export DISPLAY=`grep -oP "(?<=nameserver ).+" /etc/resolv.conf`:0.0

must be set

In [1]:
# imports
import cv2
import numpy as np
import os
import tarfile
import urllib.request
import shutil
import xlsxwriter
from datetime import datetime
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.builders import model_builder
from sort import *

In [2]:
# settings
VIDEO = 'full_360p_noAudio_24fps.mov'
FPS = 24
VIDEO_WIDTH = 640
VIDEO_HEIGHT = 360
CLASSES = {'bicycle', 'car', 'motorcycle', 'bus', 'truck'}
TRACK_HISTORY = 50

# more models can be found here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
# faster model. not as effective
MODEL_DATE = '20200713'
MODEL_NAME = 'centernet_hg104_512x512_coco17_tpu-8'

# slow model. Effective
#MODEL_DATE = '20200711'
#MODEL_NAME = 'efficientdet_d7_coco17_tpu-32'

In [3]:
# create folder structure
DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR, 'models')
for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)
        
if os.path.exists('objects'):
    shutil.rmtree('objects')
os.mkdir('objects')

if not os.path.exists('statistics'):
    os.mkdir('statistics')
    
if not os.path.exists('output_videos'):
    os.mkdir('output_videos')

In [4]:
# Download and extract model
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

# Download labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
    print('Downloading label file... ', end='')
    urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
    print('Done')

In [5]:
# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    

# Suppress TensorFlow logging (2)
tf.get_logger().setLevel('ERROR')           

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

def get_model_detection_function(model):
    """Get a tf.function for detection."""

    @tf.function
    def detect_fn(image):
        """Detect objects in image."""

        image, shapes = model.preprocess(image)
        prediction_dict = model.predict(image, shapes)
        detections = model.postprocess(prediction_dict, shapes)

        return detections, prediction_dict, tf.reshape(shapes, [-1])

    return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [6]:
# load categories
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [7]:
def get_date_and_time():
    """ Returns current date and time.
    
    This function is used to add the date and time to the output video and excel file.
    
    Return value:
    dt_string -- date and time
    """
    
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
    
    return dt_string

In [8]:
def filter_detections(detections, wanted_classes, category_index, min_thresh=0.3):
    """ Return only the detections for the wanted classes and above the minimum threshold.
    
    This function is used to filter the detections from the Tensorflow object detection api.
    The detections are filtered by the minimum threshold as well by the wanted classes.
    
    In this project it is used for the displaying and tracking of the wanted vehicle types.
    
    Keywords arguments:
    detections -- detections by the Tensorflow object detection api
    wanted_classes -- Dict, with all wanted classes who will be displayed and tracked
    category_index -- category index by the model
    min_thresh -- minimum detection score of detections which should be shown
    
    Return value:
    wanted_detections -- same as detections but the filtered ones
    """

    # extract values from the detections array
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    
    # get class ids by class name
    class_ids = []
    for value in category_index.values():
        name = value.get('name')
        id = value.get('id')
        if name in wanted_classes:
            class_ids.append(id - 1)
    
    # get detected objects with min. thresh and wanted class id
    wanted_boxes = []
    wanted_scores = []
    wanted_classes = []
    for i in range(classes.size):
        if classes[i] in class_ids and scores[i] > min_thresh:
            wanted_boxes.append(boxes[i])
            wanted_classes.append(classes[i])
            wanted_scores.append(scores[i])
    
    # prepare data as tensors
    wanted_detections = {
        'detection_boxes': tf.expand_dims(tf.convert_to_tensor(wanted_boxes), axis=0),
        'detection_classes': tf.expand_dims(tf.convert_to_tensor(wanted_classes), axis=0),
        'detection_scores': tf.expand_dims(tf.convert_to_tensor(wanted_scores), axis=0)
    }
    
    return wanted_detections

In [9]:
def object_detection(frame):
    """ Returns detections for given frame
    
    This function is used to run the object detection for every frame.
    
    Keywords arguments:
    frame -- given frame to apply the object detection
    
    Return value:
    detections -- includes detection boxes, scores, classes, ...
    """
    
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(frame, axis=0)
        
    input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    
    return detections

In [10]:
def cleanup_arrays(history, screenshots, direction_tracking, speeds, speed_tracking, track_id_score, track_id_class, stats, directions):
    """ Cleans up all arrays and dicts each frame.
    
    In order not to have too much temporary memory, each frame only the last TRACK_HISTORY track_id's are stored.
    
    Keywords arguments:
    All arrays and dicts who will be cleaned up.
    """
    
    # clean up screenshots
    screenshots = [id for id in screenshots if id >= track_id - history]
                    
    # clean up direction_tracking
    for key in list(direction_tracking.keys()):
        if key <= track_id - history:
            direction_tracking.pop(key, None)
                            
    # clean up speeds
    for key in list(speeds.keys()):
        if key <= track_id - history:
            speeds.pop(key, None)
                            
    # clean up speed_tracking
    for key in list(speed_tracking.keys()):
        if key <= track_id - history:
            speed_tracking.pop(key, None)
            
    # clean up track_id_score
    for key in list(track_id_score.keys()):
        if key <= track_id - history:
            track_id_score.pop(key, None)
            
    # clean up track_id_class
    for key in list(track_id_class.keys()):
        if key <= track_id - history:
            track_id_class.pop(key, None)
            
    # clean up stats
    stats = [id for id in stats if id >= track_id - history]
    
    # clean up directions
    for key in list(directions.keys()):
        if key <= track_id - history:
            directions.pop(key, None)

In [11]:
def speed_calculation(locations, speeds):
    """ Calculation the speed of an object.
    
    This function is used to calculate and save the speed of an object in km/h.
    
    Keywords arguments:
    locations -- x coords of the given object
    speeds -- speeds dict to save the speed by track_id
    """
    
    # Only objects that will drive completely through the speed detection area will be calculated
    if (locations[-1] >= 452 and min(locations) < 308) or (locations[-1] <= 308 and max(locations) > 452):
        # get all x coordinates in the speed detection area
        speed_tracking[track_id] = len([mark for mark in locations if (mark >= 308 and mark <=452)])
                                
        if speed_tracking[track_id] != 0:
            # speed = 4.5 meters / (amount of x coordinates in area / fps) * 3.6
            speed = 4.5/(speed_tracking[track_id]/FPS)*3.6
            
            # false calculation
            if speed < 100.0:
                speeds[track_id] = speed

In [12]:
def display_directions(frame, locations, rectangle, directions, track_id):
    """ Display the directions with an arrow.
    
    To display the directions of an object it will check the last three locations of it.
    It is displayed as an arrow on the right or left side of the objects detection box.
    
    Keywords arguments:
    frame -- frame where the direction should be displayed
    locations -- x coords of the object
    rectangle -- rectangle of the object
    directions -- directions dict to save the directions of the object
    track_id -- current object
    
    Return value:
    frame -- frame with directions
    """
    x1, x2, y1, y2 = rectangle
    
    # get height of arrow depending on the rectangle size
    y_startPoint = int(y1+((y2-y1)/2))
                        
    # object moves to the right
    if locations[-1] > (locations[-2] and  locations[-3]):
        cv2.arrowedLine(frame, (x2+5, y_startPoint), (x2+25, y_startPoint), (0,255,0), thickness=1, tipLength=0.2)
        directions[track_id] = 'right'
                            
    # object moves to the left
    elif locations[-1] < (locations[-2] and locations[-3]): # left
        cv2.arrowedLine(frame, (x1-5, y_startPoint), (x1-25, y_startPoint), (0,255,0), thickness=1, tipLength=0.2)
        directions[track_id] = 'left'

    return frame

In [13]:
def display_speeds(frame, track_id, speeds, rectangle):
    """ Display the speed of an object.
    
    The speed will be display under the object as soon as the speed is calculated.
    
    Keywords arguments:
    frame -- frame where the speed should be displayed
    track_id -- current object
    speeds -- speeds dict to get the speed
    rectangle -- rectangle of the object
    
    Return value:
    frame -- frame with speeds
    """
    
    x1, x2, y1, y2 = rectangle
    
    # display speed below the track rectangle
    if track_id in speeds:
        text = "{0:.1f}km/h".format(speeds[track_id])
        (speedTextSize_width, speedTextSize_height), baseline_speed = cv2.getTextSize(text, font, fontScale, lineType)
        speedTextSize = (speedTextSize_width, speedTextSize_height)
        speedTextX = int(x1+(x2-x1)/2 - (speedTextSize[0] / 2))
        speedTextY = int(y2 + (speedTextSize[1] / 2) + speedTextSize_height/2 + 7)
        bottomLeftCornerOfSpeedText = (speedTextX, speedTextY)
                        
        cv2.rectangle(frame, (speedTextX - baseline_speed, speedTextY - speedTextSize[1] - baseline_speed), (speedTextX + speedTextSize[0] + baseline_speed, speedTextY + baseline_speed), (0,255,0), thickness=-1)
        cv2.putText(frame, text, (bottomLeftCornerOfSpeedText), font, fontScale, fontColor, lineType)
        
    return frame

In [14]:
def display_tracking_detection(frame, x_coord, rectangle, track_id_score, track_id_class, track_id):
    """ Display data by the object detection.
    
    Display the score and class of an object by the object detection. It is displayed above the rectangle by the tracking.
    
    Keywords arguments:
    frame -- frame where the object detection should be displayed
    x_coord -- used to shoose color of rectangle
    rectangle -- rectangle of tracking
    track_id_score -- scores for each track_id
    track_id_class -- classes for each track_id
    track_id -- current track_id
    
    Return value:
    frame -- frame with scores and classes
    """
    
    x1, x2, y1, y2 = rectangle
    
    # display score and class
    # get middle of top rectangle
    middle = (int(x1 + ((x2-x1)/2)), y1)
    
    score_text = '{}%'.format(track_id_score[track_id])
    class_text = track_id_class[track_id]
        
    # align score text
    (scoreTextSize_width, scoreTextSize_height), baseline_score = cv2.getTextSize(score_text, font, fontScale, lineType)
    scoreTextSize = (scoreTextSize_width, scoreTextSize_height)
    scoreTextX = int(middle[0] - (scoreTextSize[0] / 2))
    scoreTextY = int(middle[1] + (scoreTextSize[1] / 2) -12)
    bottomLeftCornerOfScoreText = (scoreTextX, scoreTextY)
        
    # align class name text
    (classTextSize_width, classTextSize_height), baseline_class = cv2.getTextSize(class_text, font, fontScale, lineType)
    classTextSize = (classTextSize_width, classTextSize_height)
    classTextX = int(middle[0] - (classTextSize[0] / 2))
    classTextY = int(middle[1] + (classTextSize[1] / 2) -30)
    bottomLeftCornerOfClassText = (classTextX, classTextY)
        
    # display score
    cv2.rectangle(frame, (scoreTextX - baseline_score, scoreTextY - scoreTextSize[1] - baseline_score), (scoreTextX + scoreTextSize[0] + baseline_score, scoreTextY + baseline_score), (0,255,0), thickness=-1)
    cv2.putText(frame, score_text, bottomLeftCornerOfScoreText, font, fontScale, fontColor, lineType)
        
    # display class name
    cv2.rectangle(frame, (classTextX - baseline_class, classTextY - classTextSize[1] - baseline_class), (classTextX + classTextSize[0] + baseline_class, classTextY + baseline_class), (0,255,0), thickness=-1)
    cv2.putText(frame, class_text, bottomLeftCornerOfClassText, font, fontScale, fontColor, lineType)
    
    # display tracking rectangle. Red if it is in the speed detection area
    if x_coord >= 308 and x_coord <= 452:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), thickness=1)
    else:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), thickness=1)
        
    return frame

In [15]:
def area(a, b):
    """ Calculating the overlapping area of two rectangles.
    
    This function is used to calculate the overlapping area of two rectangles.
    
    Keywords arguments:
    a -- first rectangle
    b -- second rectangle
    
    Return value:
    area -- Area of the two overlapping rectangles
    """
    
    ax1, ax2, ay1, ay2 = a
    bx1, bx2, by1, by2 = b
    
    if ax1 >= ax2:
        axmax = ax1
        axmin = ax2
    else:
        axmax = ax2
        axmin = ax1
        
    if bx1 >= bx2:
        bxmax = bx1
        bxmin = bx2
    else:
        bxmax = bx2
        bxmin = bx1
    
    if ay1 >= ay2:
        aymax = ay1
        aymin = ay2
    else:
        aymax = ay2
        aymin = ay1
        
    if by1 >= by2:
        bymax = by1
        bymin = by2
    else:
        bymax = by2
        bymin = by1
    
    dx = min(axmax, bxmax) - max(axmin, bxmin)
    dy = min(aymax, bymax) - max(aymin, bymin)
    if (dx>=0) and (dy>=0):
        return dx*dy
    else:
        return 0

In [16]:
def movement_detection(frame, fgbg, gaussian_kernel=(7,7), dilation_kernel=(11,11), minContourArea=1200):
    """ Movement detection of objects of a certain size.
    
    This function detects any kind of movement in a frame. Only movement of a certain size will be detected.
    Only the moving objects will be shown and the rest of the image will be black.
    
    In this project it is used to get all moving vehicles and objects of a
    video to differentiate between moving and standing vehicles and objects.
    
    Keywords arguments:
    frame -- frame to apply the movement detection
    fgbg -- foreground/background subtractor
    gaussian_kernel -- kernel size for the gaussian blur (default (7,7))
    dilation_kernel -- kernel size for the dilation (default (11,11))
    minContourArea -- minimum of contour area to be shown (default 1200)
    
    Return value:
    movement_frame -- same as frame, but all spots without moving objects are blacked out
    """
    
    # grayscale
    gray_frame = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)
    
    # blurring with gaussian blur
    blur_frame = cv2.GaussianBlur(src=gray_frame, ksize=gaussian_kernel, sigmaX=0)
    
    # applying background subtraction
    fgmask = fgbg.apply(blur_frame)
    
    # dilation of the foreground mask
    dilated_frame = cv2.dilate(src=fgmask, kernel=np.ones(dilation_kernel,np.uint8), iterations=3)
    
    # returning all contours
    contours, _ = cv2.findContours(image=dilated_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    
    # create mask with same size and shape as the frame
    mask = np.zeros_like(frame)
    
    # Draw rectangles around contours
    for contour in contours:
            
        # Skipping small contours
        if cv2.contourArea(contour) < minContourArea:
            continue
                
        # Returning corners of the contour. Doesn't consider the rotation of the object.
        (x, y, w, h) = cv2.boundingRect(contour)

        # draw white filled contour on the mask
        cv2.rectangle(img=mask, pt1=(x, y), pt2=(x+w, y+h), color=(255,255,255), thickness=-1)

    # add mask and frame. Only show white areas from mask on the frame.
    movement_frame = cv2.bitwise_and(frame, mask)
    
    return movement_frame

In [17]:
def assign_track_with_object(filtered_detections, track_id):
    """ Assign the score and class to tracked object.
    
    By calculating the area of both rectangle. The rectangles with the biggest area are the same object.
    
    Keywords arguments:
    filtered_detections -- detections to compare
    track_id -- current track_id
    """
    
    # assing tracking objects with object detection objects
    boxes = filtered_detections['detection_boxes'][0].numpy()
    areas = []
    
    for i in range(len(boxes)):
        # get rectangle coordinates
        bx1 = int(VIDEO_WIDTH * boxes[i][1])
        by1 = int(VIDEO_HEIGHT * boxes[i][0])
        bx2 = int(VIDEO_WIDTH * boxes[i][3])
        by2 = int(VIDEO_HEIGHT * boxes[i][2])
        rectangle2 = (bx1, bx2, by1, by2)
                        
        areas.append(area(rectangle, rectangle2))
                        
    i = areas.index(max(areas))
                                        
    scores = filtered_detections['detection_scores'][0].numpy()
    classes = filtered_detections['detection_classes'][0].numpy()
                    
    track_id_score[track_id] = int(scores[i] * 100)
    track_id_class[track_id] = category_index[classes[i] + 1]['name']

In [18]:
# get time and date
dt_string = get_date_and_time()

# load video
video = cv2.VideoCapture(VIDEO)

# open video to save the output
result = cv2.VideoWriter('output_videos/detection_{}.mp4'.format(dt_string), cv2.VideoWriter_fourcc(*'mp4v'), 24.0, (640, 360), True)

# create background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=800, detectShadows=False, varThreshold=100)

# create tracker
mot_tracker = Sort()

# create temporary dicts and lists
direction_tracking = {}
speed_tracking = {}
screenshots = []
speeds = {}
track_id_score = {}
track_id_class = {}
stats = []
directions = {}
row = 1
amount = {'bicycle': 0, 'car': 0, 'motorcycle': 0, 'bus': 0, 'truck': 0}

# Create excel file
workbook = xlsxwriter.Workbook('statistics/stats_{}.xlsx'.format(dt_string))
worksheet_objects = workbook.add_worksheet('Objects')
worksheet_amount = workbook.add_worksheet('Amount')
bold = workbook.add_format({'bold': True})

# add header to Objects worksheet
worksheet_objects.write(0,0, 'track_id', bold)
worksheet_objects.write(0,1, 'class', bold)
worksheet_objects.write(0,2, 'speed in km/h', bold)
worksheet_objects.write(0,3, 'direction', bold)

# define font
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.4
fontColor = (0,0,0)
lineType = 1
    
while(video.isOpened()):
        ret, frame = video.read()
        
        # get movement frame
        movement_frame = movement_detection(frame, fgbg)
        
        # get detections
        detections = object_detection(movement_frame)
        
        # get relevant detections
        filtered_detections = filter_detections(detections, {'bicycle', 'car', 'motorcycle', 'bus', 'truck'}, category_index, 0.4)
        
        # check if relevant detections are found
        if filtered_detections['detection_boxes'].shape != (1, 0):
            
            # update tracker
            track_bbs_ids = mot_tracker.update(filtered_detections['detection_boxes'][0])
            
            # check if tracker updated
            if track_bbs_ids.size != 0:
                
                for track in track_bbs_ids:
                    
                    # extract values from tracking list
                    track_id = int(track[4])
                    x1 = int(VIDEO_WIDTH*track[1])
                    y1 = int(VIDEO_HEIGHT*track[0])
                    x2 = int(VIDEO_WIDTH*track[3])
                    y2 = int(VIDEO_HEIGHT*track[2])
                    middle = (int(x1 + ((x2-x1)/2)), int(y1 + ((y2-y1)/2)))
                    rectangle = (x1, x2, y1, y2)
                    x_coord = middle[0]
                    
                    assign_track_with_object(filtered_detections, track_id)
                    
                    # add x coordinates of tracked object to track_id
                    if track_id in direction_tracking:
                        direction_tracking[track_id].append(x_coord)
                    else:
                        direction_tracking[track_id] = [x_coord]
                        
                    # objects has to be tracked at least three times to analyze direction and speed
                    if direction_tracking.get(track_id) != None and len(direction_tracking.get(track_id)) >= 3:
                        
                        # get all x coordinates of tracked object
                        locations = direction_tracking.get(track_id)
                        
                        frame = display_directions(frame, locations, rectangle, directions, track_id)
                                        
                        speed_calculation(locations, speeds)

                    display_speeds(frame, track_id, speeds, rectangle)    
                    
                    display_tracking_detection(frame, x_coord, rectangle, track_id_score, track_id_class, track_id)

                    # save data for current frame
                    if track_id in directions:
                        if ((x_coord > 452 and directions[track_id] == 'right') or (x_coord < 308 and directions[track_id] == 'left')) and track_id not in stats:
                            stats.append(track_id)
                            if track_id in speeds:
                                cv2.imwrite('objects/object_{}.jpg'.format(track_id), movement_frame[y1:y2, x1:x2])
                                worksheet_objects.write(row, 0, track_id)
                                worksheet_objects.write(row, 1, track_id_class[track_id])
                                worksheet_objects.write_number(row, 2, speeds[track_id])
                                worksheet_objects.write(row, 3, directions[track_id])
                                row += 1
                                amount[track_id_class[track_id]] += 1
                            
                    cleanup_arrays(TRACK_HISTORY, screenshots, direction_tracking, speeds, speed_tracking, track_id_score, track_id_class, stats, directions)

        # marking speed detection area
        cv2.line(frame, (308, 0), (308, VIDEO_HEIGHT), (255, 102, 0), thickness=1)
        cv2.line(frame, (452, 0), (452, VIDEO_HEIGHT), (255, 102, 0), thickness=1)
        
        # display amount of classes
        increaser = 0
        for key, value in amount.items():
            cv2.putText(frame, key + ': ' + str(value), (0,355 + increaser), font, fontScale, (0,255,0), lineType)
            increaser -= 10
        
        # display frame
        cv2.imshow('frame', cv2.resize(frame, (1920, 1080)))
        result.write(frame)
        
        # Press q to exit video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# save amount
count = 1
worksheet_amount.write(0, 0, 'object', bold)
worksheet_amount.write(0, 1, 'amount', bold)
for key, value in amount.items():
    worksheet_amount.write(count, 0, key)
    worksheet_amount.write_number(count, 1, value)
    count += 1

workbook.close()
video.release()
result.release()
cv2.destroyAllWindows()